In [7]:
# -*- coding: utf-8 -*-

# LSTM for sequence classification in the IMDB dataset
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pandas as pd
import nltk
import itertools


In [5]:
def get_class(x):
    #print x['tweet;class']
    classe = x['tweet;class'].split(';')[1]
    return classe

def get_text(x):
    #print x['tweet;class']
    text = x['tweet;class'].split(';')[0]
    return text

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import islice
import operator

def get_most_relevant(a,b,n):
    s=[]
    corpus = [a,b]
    vectorizer = TfidfVectorizer(min_df=1)
    X = vectorizer.fit_transform(corpus)
    idf = vectorizer.idf_
    x= dict(zip(vectorizer.get_feature_names(), idf))
    
    sorted_dic = sorted(x.items(), key=operator.itemgetter(1), reverse=True)
    elements=sorted_dic[:n]
    for w,tf in elements:
        s.append(w)
    return elements
        
    
a= "This is very strange"
b="This is very nice"
get_most_relevant(a,b,2)

[(u'strange', 1.4054651081081644), (u'nice', 1.4054651081081644)]

In [208]:
vocab[0]

(u'#', 4273)

In [9]:
percent_train=0.9
vocabulary_size = 10000
max_review_length = 30
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

df = pd.read_csv('olympic_2012_pos_neg.csv')
df['class']= df.apply(get_class,axis=1)
df['text']= df.apply(get_text,axis=1)

sentences=[]
classes=[]
lens=[]
for r in df.T.to_dict().values():
    text = (str(r['text']).decode('iso-8859-1').lower())
    sentences.append(text)
    lens.append(len(text.split()))
    classes.append(r['class'])

#max_review_length = int(np.mean(lens))
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())    

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)

##vocab by tfidf
#vocab_p=""
#vocab_n=""
#for i in xrange(len(classes)):
#    if classes[i]== "positive":
#        vocab_p = vocab_p + " " + sentences[i]
#    else:
#        vocab_n = vocab_n + " " + sentences[i]#
#
#vocab= get_most_relevant(vocab_p,vocab_n, vocabulary_size-1)


index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])
 
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
 
print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]

Y=[]
for i in xrange(len(classes)):
    if classes[i]=="positive":
        Y.append(1)
    else:
         Y.append(0)
            
            
# Create the training data
sidx = np.random.permutation(len(tokenized_sentences))
n_train = int(np.round(len(tokenized_sentences) * (percent_train)))
valid_set_x = [tokenized_sentences[s] for s in sidx[n_train:]]
y_valid = [Y[s] for s in sidx[n_train:]]
train_set_x = [tokenized_sentences[s] for s in sidx[:n_train]]
y_train = [Y[s] for s in sidx[:n_train]]
    
    

X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in train_set_x])
X_valid = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in valid_set_x])

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_valid = sequence.pad_sequences(X_valid, maxlen=max_review_length)



Found 11803 unique words tokens.
Using vocabulary size 10000.
The least frequent word in our vocabulary is 'sarahwbal' and appeared 1 times.

Example sentence: '  - enough said!! woiii!! '

Example sentence after Pre-processing: '[u'-', u'enough', u'said', u'!', u'!', 'UNKNOWN_TOKEN', u'!', u'!']'


In [13]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=6, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_valid, y_valid, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#x= np.asarray([[word_to_index[w] for w in "not bad".split()]])
#xpad=sequence.pad_sequences(x, maxlen=max_review_length)
#model.predict(xpad)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 30, 32)        320000      embedding_input_4[0][0]          
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 100)           53200       embedding_4[0][0]                
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 1)             101         lstm_4[0][0]                     
Total params: 373301
____________________________________________________________________________________________________
None
Epoch 1/6
3632/3632 [==============================] - 5s - loss: 0.5763 - acc: 0.7373     
Epoch 2/6
3632/3632 [==============================] - 5s - loss: 0.3491 -